### Imports

In [ ]:
import os
import pandas as pd
import numpy as np
from functools import reduce

In [ ]:
data_dir = "../base_data"
new_data_dir = "new_data"

**Data to use in the beginning**

In [ ]:
columns_to_keep_matches = {
    'fixtures.csv': ['eventId', 'homeTeamId', 'awayTeamId','leagueId', 'homeTeamWinner', 'awayTeamWinner', 'homeTeamScore', 'awayTeamScore', 'date'],
}

In [ ]:
dfs_matches = []

for filename, columns in columns_to_keep_matches.items():
    path = os.path.join(data_dir, filename)
    df = pd.read_csv(path, usecols=columns)
    dfs_matches.append(df)
   

In [ ]:
merged_df_matches = reduce(lambda left, right: pd.merge(left, right, on='eventId', how='inner'), dfs_matches)

# Generate ordered column list automatically
ordered_columns_m = []
for cols in columns_to_keep_matches.values():
    for col in cols:
        if col not in ordered_columns_m:
            ordered_columns_m.append(col)

# Reorder columns safely
merged_df_matches = merged_df_matches[[col for col in ordered_columns_m if col in merged_df_matches.columns]]
merged_df_matches = merged_df_matches.dropna()

**Using only matches from Bundesliga(GER1) with league Id = 720 and after 30.8.2024, which is the season start date**

In [ ]:
merged_df_matches = merged_df_matches[merged_df_matches['date'] > '2024-08-30']
merged_df_matches = merged_df_matches[merged_df_matches['leagueId'] == 720]
merged_df_matches['date'] = pd.to_datetime(merged_df_matches['date'])
merged_df_matches

In [ ]:
merged_df_matches = merged_df_matches.groupby(['eventId']).agg(
homeTeamId=pd.NamedAgg(column="homeTeamId", aggfunc="first"),
awayTeamId=pd.NamedAgg(column="awayTeamId", aggfunc="first"),
leagueId=pd.NamedAgg(column="leagueId", aggfunc="first"),
homeTeamScore=pd.NamedAgg(column="homeTeamScore", aggfunc="first"),
awayTeamScore=pd.NamedAgg(column="awayTeamScore", aggfunc="first"),
homeTeamWinner=pd.NamedAgg(column="homeTeamWinner", aggfunc="first"),
awayTeamWinner=pd.NamedAgg(column="awayTeamWinner", aggfunc="first"),
date=pd.NamedAgg(column="date", aggfunc="first"),

).sort_values(by=['leagueId', 'date']).reset_index()

**Mapping the league code to official short name of the league**

In [ ]:
# Define custom league name mapping
league_name_map = {
    720: "GER1"
}

# Replace leagueId with custom league names
merged_df_matches['leagueId'] = merged_df_matches['leagueId'].map(league_name_map)

**Output df to csv**

In [ ]:
output_path_matches = os.path.join(new_data_dir, 'database_matches.csv')

os.makedirs(os.path.dirname(output_path_matches), exist_ok=True)

merged_df_matches.to_csv(output_path_matches, index=False)

print(f"Merged CSV saved to {output_path_matches}")
